## Functionality: visualization of functional connectivity

<ul>
    <li>142 channels = 1-96 channels (M1) + 97-100(not used) + 101-132(ThaSMA) + 133-146 (GP/STN)</li>
    <li>load animetafolder/pinky_locThaSMA.csv for the details of ThaSMA </li>
</ul>

## Import Packages 

In [3]:
import os, sys
import numpy as np

import pickle

# import_nbmodule used for import package in .ipynb
import import_nbmodule

# extract the exp folder path
currfolder = os.getcwd()
codefolder = currfolder[0 : currfolder.find('code')+len('code')]

# add path the exp folder
sys.path.append(codefolder)

# import animal_folder function in util/folder_extract.pynb 
from util.folder_extract import exp_subfolders


# import graphVisual in connAnalyTool/graphVisual.pynb 
from connAnalyTool import graphVisual


from igraph.drawing.text import TextDrawer
import cairo

importing Jupyter notebook from /home/lingling/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/util/folder_extract.ipynb


## load the ciCOH matrix

### Load Beta band ciCOH
1. dic_ciCOH.keys() = ['ciCOH_beta1_normal', 'ciCOH_beta1_mild', 'ciCOH_beta2_normal', 'ciCOH_beta2_mild']

In [8]:
def ciCOHBetabandLoad(betaciCOHfilefolder):
    """
        load beta band ciCOH
        
        
        @return ciCOH_beta1_normal, ciCOH_beta1_mild: bata1 band ciCOH values in normal and mild conditions
                                                      (ndarray: nchns * nchns * ntrials)
        
        @return ciCOH_beta2_normal, ciCOH_beta2_mild: bata2 band ciCOH values in normal and mild conditions 
                                                      (ndarray: nchns * nchns * ntrials)
        
    """

    # file storing the ciCOH results
    ciCOHfile = os.path.join(betaciCOHfilefolder, 'res_ciCOH_beta.pickle')

    # load res_ciCOH
    res_ciCOH_beta = pickle.load(open(ciCOHfile, "rb" ))

    # ciCOH_beta1_normal, ciCOH_beta1_mild
    ciCOH_beta1_normal, ciCOH_beta1_mild = res_ciCOH_beta['ciCOH_beta1_normal'], res_ciCOH_beta['ciCOH_beta1_mild']

    # ciCOH_beta2_normal, ciCOH_beta2_mild
    ciCOH_beta2_normal, ciCOH_beta2_mild = res_ciCOH_beta['ciCOH_beta2_normal'], res_ciCOH_beta['ciCOH_beta2_mild']
    
    # return
    return ciCOH_beta1_normal, ciCOH_beta1_mild, ciCOH_beta2_normal, ciCOH_beta2_mild

In [ ]:
betaciCOHfilefolder = os.path.join(pipelinefolder, '1_dataAnaly', 'FCAnaly', '0_calcciCOH_beta', 'out')

# load beta1/beta2 ciCOH in normal and mild conditions
ciCOH_beta1_normal, ciCOH_beta1_mild, ciCOH_beta2_normal, ciCOH_beta2_mild = ciCOHBetabandLoad(betaciCOHfilefolder)

### Load alpha band ciCOH
1. dic_ciCOH.keys() = ['ciCOH_alpha_normal', 'ciCOH_alpha_mild']

In [ ]:
def ciCOHAlphabandLoad(alphaciCOHfilefolder):
    """
        load alpha band ciCOH
        
        @return ciCOH_alpha_normal,  ciCOH_alpha_mild: alpha band ciCOH values in normal and mild 
                                                       (ndarray: nchns * nchns * ntrials) 
    """

    # file storing the ciCOH results
    ciCOHfile = os.path.join(alphaciCOHfilefolder, 'res_ciCOH_alpha.pickle')

    # load res_ciCOH_alpha
    res_ciCOH_alpha = pickle.load(open(ciCOHfile, "rb" ))

    # ciCOH_alpha_normal, ciCOH_alpha_mild
    ciCOH_alpha_normal, ciCOH_alpha_mild = res_ciCOH_alpha['ciCOH_alpha_normal'], res_ciCOH_alpha['ciCOH_alpha_mild']
    
    # return
    return ciCOH_alpha_normal,  ciCOH_alpha_mild

In [ ]:
alphaciCOHfilefolder = os.path.join(pipelinefolder, '1_dataAnaly', 'FCAnaly', '0_calcciCOH_alpha', 'out')

# load beta1/beta2 ciCOH in normal and mild conditions
ciCOH_beta1_normal, ciCOH_beta1_mild, ciCOH_beta2_normal, ciCOH_beta2_mild = ciCOHBetabandLoad(alphaciCOHfilefolder)

### Load simulated coordinates

In [15]:
def simcoordLoad(simcoordfilefolder):
    """
        load simulated coordinates of areas
        
        @return coord_area: simulated x, y coordinates in areas (dict, e.g. coord_area["M1"]: ndarray nchns * 2)
        
    """
    # file storing the simulated coordinates
    coordfile = os.path.join(simcoordfilefolder, 'coord_simulated.pickle')

    # load simulated coordinates
    coord_area = pickle.load(open(coordfile, "rb" ))
    
    # return
    return coord_area

In [16]:
simcoordfilefolder = os.path.join(pipelinefolder, '1_dataAnaly', 'FCAnaly', '0_simulatedLocs', 'out')

simcoordLoad(simcoordfilefolder)

coord_area = simcoordLoad(simcoordfilefolder)

### Set area and coordinates of all electrodes 

In [ ]:
def setAreaPosofElec(coord_area):
    """
        set the area and coordinates of all electrodes
        
        @param coord_area: x, y coordinates in areas (dict, e.g. coord_area["M1"]: ndarray nchns * 2)
        
        
        
        @return elec_area: brain area of all electrodes ()
        
        @return elec_coord: coordinates of all electrodes ()
        
    """
    
    # coordinates of all electrodes
    elec_coord = np.empty((0,2))
    # area of all electrodes
    elec_area = []
    for key in coord_area.keys():
        if 'coord_' in key:

            # coordinates of key
            coord_key = coord_area[key]

            # area sequence: M1 * 96, ThaSMA *32, STN * 7 and GP * 7
            elec_area = elec_area + [key[len('coord_'):]] * len(coord_key)

            # append to coordinates of electrodes elec_coord
            elec_coord = np.append(elec_coord, coord_key, axis = 0)

            # del
            del coord_key
            
    return elec_area, elec_coord

In [ ]:
# M1 part
brainarea = 'M1'
coord, labels = coord_area['coord_' + brainarea], coord_area['labels_' + brainarea]

# ThaSMA part
brainarea = 'ThaSMA'
coord, labels = coord_area['coord_' + brainarea], coord_area['labels_' + brainarea]

# STN part
brainarea = 'STN'
coord, labels = coord_area['coord_' + brainarea], coord_area['labels_' + brainarea]

# GP part
brainarea = 'GP'
coord, labels = coord_area['coord_' + brainarea], coord_area['labels_' + brainarea]

In [ ]:
# ThaSMA part
brainarea = 'ThaSMA'
coord, labels = coord_area['coord_' + brainarea], coord_area['labels_' + brainarea]

In [ ]:
# coordinates of all electrodes
elec_coord = np.empty((0,2))
# area of all electrodes
elec_area = []


key = 'coord_M1'
coord_key = coord_area[key]

# area sequence: M1 * 96, ThaSMA *32, STN * 7 and GP * 7
elec_area = elec_area + [key[len('coord_'):]] * len(coord_key)

# append to coordinates of electrodes elec_coord
elec_coord = np.append(elec_coord, coord_key, axis = 0)

In [ ]:
# areachns: dictionary storing the channel number for each brain area, e.g areachns['lSMA'] = [113, 118, 123,  124]
chns_ThaSMA = dict()

for i, key in enumerate(df[0]):
    
    # split df[1][i] using , and transfer to int
    listchns = [int(strchn) for strchn in df[1][i].split(",")]
       
    # 97 - 128 corresponding to 101-132
    chns_ThaSMA[key] = [chn - 4 for chn in listchns]

In [ ]:
# chnsInf dictionary with column name: chni, brainarea, coordinates
chnsInf = dict()

# number of channels in M1
nchn_M1 = 96
chni = np.linspace(0, 95, num = nchn_M1, dtype = np.uint8)
chnsInf['chni'] = chni
chnsInf['brainArea'] = ['M1'] * nchn_M1
chnsInf['coordinates'] = coord_area['coord_M1']


# Stop Here!!!!
Input the chnsInf for ThaSMA

print(chnsInf)

In [ ]:
chns = []
ThaSMAarea = []
for key in chns_ThaSMA:
    chns = chns + chns_ThaSMA[key]
    
    ThaSMAarea = ThaSMAarea + [key] * len(chns_ThaSMA[key])
    
chns = np.asarray(chns)
print(chns)
print(ThaSMAarea)

In [ ]:
print(np.argsort(np.asarray(chns)))

In [ ]:
chns = chns_ThaSMA['lSMA']
print(elec_coord)

In [ ]:
print(labels)
for label in labels:
    area = re.match(r"([a-z]+)([0-9]+)", label, re.I)
    print(area.group(1))

### set plot text

In [ ]:
def plottexts(fontsize_area = 20):
    """
        @param fontsize_area: font size for ploting the text 
        
        @return texts_area: a dictionary with key is the text, key value is the [x, y, fontsize]
    """
    # coordinates of plot area text (e.g "M1")
    texts_area = dict()
    texts_area["M1"],  texts_area["ThaSMA"] = [0, 180], [325, 180]
    texts_area["STN"], texts_area["GP"] = [15, 530], [275,530] 

    # add fontsize to each key
    for key in texts_area:
        texts_area[key] =  texts_area[key] + [fontsize_area]
        
        
    return texts_area

## visualFC function

In [ ]:
def visualFC(weight, elec_area, elec_coord, lowweight = 0.1, texts = None):
    
    """
        @param weight: functional connectivity matrix (nchns * nchns)
        
        @param elec_area: a list storing the area for each electrode (len(elec_area) = nchns)
        
        @param elec_coord: the 2d coordinates of all electrodes (numpy.ndarray, nchns * 2)
        
        @param texts: a dictionary storing the text to be printed with text[key]: [x,y, fontsize]
        e.g texts["M1"] = [0, 180, 30]
        
        
        @return igplot: an igraph.drawing.Plot object
        
    """
    

    # weight > 0.1 as 
    weight[np.where(weight<lowweight)] =0
    
    # create graph
    graph = graphVisual.graph_create(weight, vsarea = elec_area, vscoord = elec_coord)

    # set graph visualization style
    visual_style = graphVisual.graph_style(graph)
    
    
    igplot = graphVisual.graph_plot(graph, visual_style, texts)
    
    return igplot

In [ ]:
def bandciCOHVisulaSave(bandciCOH, bandname, condition, savefolder, elec_area, elec_coord, texts_area=None):
    """
        @param bandciCOH: ciCOH weight (nchns * nchns * ntrials)
        
        @param bandname: 'beta1', 'beta2' or 'alpha'
        
        @param condition: 'normal' or 'mild'
        
        @param savefolder:
        
        @param elec_area: 
        
    """
    
    weight = abs(np.mean(bandciCOH, axis = 2))
    
    if texts_area is not None:
        # copy the texts_area
        texts = texts_area.copy()
    else:
        texts = dict()
        
    # add key title with [x, y, fontsize]
    fontsize_title = 30
    texts[bandname + ":" + condition] = [175, 25, fontsize_title]
     
        
    # plot FC in beta1:normal
    igplot = visualFC(weight, elec_area, elec_coord, lowweight = 0.1, texts = texts)
    
    
    # savefile
    savefile = os.path.join(savefolder, 'ciCOH_' + bandname + condition + '.png')
    
    # save
    igplot.save(savefile)
    
    # show graph
    return igplot

## Main Function

In [ ]:
anidatfolder, aniresfolder, animetafolder = animal_folder(animal = 'Pinky')

# load simulated coordinates of areas
coord_area = simcoordLoad(animetafolder)

elec_area, elec_coord = setAreaPosofElec(coord_area)

In [ ]:
print(elec_area)

In [ ]:
import pandas as pd

# read the channel information for ThaSMA
df = pd.read_csv(os.path.join(animetafolder, 'pinky_chnInfThaSMA.csv'), header = None)

In [ ]:
print(areachns)

In [ ]:
print(animetafolder)
coordfile = os.path.join(animetafolder, 'coord_simulated.pickle')

# load simulated coordinates
coord_area = pickle.load(open(coordfile, "rb" ))

print(coord_area['labels_ThaSMA'])
print(coord_area['coord_ThaSMA'])

In [ ]:
ciCOH_M1M1 = ciCOH_beta1_normal

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

for key in coord_area:
    
    if "coord" in key:
        ax.scatter(coord_area[key][:,0], coord_area[key][:,1])
        
ax.set_aspect('equal')

In [ ]:
# extract animal folder
anidatfolder, aniresfolder, animetafolder = animal_folder(animal = 'Pinky')

# load simulated coordinates of areas
coord_area = simcoordLoad(animetafolder)

# set the area and coordinates of all electrodes
elec_area, elec_coord = setAreaPosofElec(coord_area)

# save folder for FC visualization
savefolder_fcvisual = os.path.join(aniresfolder, 'FCVisual')

# load texts_area
texts_area = plottexts(fontsize_area = 20)

### Beta band FC Visualization

In [ ]:
_, _, pipelinefolder, _ = exp_subfolders()

In [ ]:
# beta1 normal
bandciCOHVisulaSave(ciCOH_beta1_normal, 
                    bandname = "beta1", condition = "normal", 
                    savefolder = savefolder_fcvisual, 
                    elec_area = elec_area, elec_coord = elec_coord, 
                    texts_area = texts_area)

# beta1 mild
bandciCOHVisulaSave(ciCOH_beta1_mild, 
                    bandname = "beta1", condition = "mild", 
                    savefolder = savefolder_fcvisual, 
                    elec_area = elec_area, elec_coord = elec_coord, 
                    texts_area = texts_area)

In [ ]:
# beta2 normal
bandciCOHVisulaSave(ciCOH_beta2_normal, 
                    bandname = "beta2", condition = "normal", 
                    savefolder = savefolder_fcvisual, 
                    elec_area = elec_area, elec_coord = elec_coord, 
                    texts_area = texts_area)

# beta2 mild
bandciCOHVisulaSave(ciCOH_beta2_mild, 
                    bandname = "beta2", condition = "mild", 
                    savefolder = savefolder_fcvisual, 
                    elec_area = elec_area, elec_coord = elec_coord, 
                    texts_area = texts_area)

### Alpha band FC Visualization

In [ ]:
# alpha normal
bandciCOHVisulaSave(ciCOH_alpha_normal, 
                    bandname = "alpha", condition = "normal", 
                    savefolder = savefolder_fcvisual, 
                    elec_area = elec_area, elec_coord = elec_coord, 
                    texts_area = texts_area)

# beta2 mild
bandciCOHVisulaSave(ciCOH_alpha_mild, 
                    bandname = "alpha", condition = "mild", 
                    savefolder = savefolder_fcvisual, 
                    elec_area = elec_area, elec_coord = elec_coord, 
                    texts_area = texts_area)